Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [RIDE]().
<hr>

# Rhode Island Public School's Poverty Proficency

This Notebook will combine data from RI Dept. of Education on enrollment of students recieving FRL and the proficency metrics during the 2016-2017 school year. The data will then be used to make scatter plots in tableau seeing how poverty and profiency may align.

In [1]:
import pandas as pd
import csv

In [2]:
df = pd.read_csv('D:/Files/datasets/education/final_clean.csv')

# filter proficency data for wanted columns
keep_vals = [
    '11th Grade Science', '4th Grade Science', '8th Grade Science',
    '3rd Grade English', '3rd Grade Math', '4th Grade English',
    '4th Grade Math', '5th Grade English', '5th Grade Math',
    '6th Grade English', '6th Grade Math', '7th Grade English',
    '7th Grade Math', '8th Grade English', '8th Grade Math',
    '9th Grade English'
]

# get wanted values and make grade column
df = df[df['subject'].isin(keep_vals)].copy()
df['grade'] = df['subject'].str[:1]
df.loc[df['grade'] == '1', 'grade'] = '11'

# wanted year
df = df[df['year'] == 2016]

# filter for final dataset to merge
cols = ['School', 'level', 'subject', 'year', 'value', 'grade', 'District', 'region']
df = df[cols]

renamer = {'School':'school', 'District':'district', 'value':'percent_proficient'}
df = df.rename(columns=renamer)

# remove no data on proficiency
df = df[~df['percent_proficient'].isnull()]

# get subject instead of subject + grade
df['subject'] = df['subject'].apply(lambda x: x.split(' '))
df['subject'] = df['subject'].apply(lambda x: ' '.join(x[-1:]))

# remove excess spacing
df['school'] = df['school'].str.strip()

# convert grade to string for join
df['grade'] = df['grade'].astype(str)

# convert names to match FRL dataset
school_translate = dict(csv.reader(open("./school-name-translate.csv")))

df['school2'] = df['school'].map(school_translate)
df.loc[df['school2'].isnull(),'school2'] = df['school']
df = df.drop(['school'], axis=1)
df = df.rename(columns={'school2': 'school'})

prof_df = df.copy()

In [3]:
frl_df = pd.read_csv('D:/Files/datasets/education/percent_frl_2017.csv')

# filter wanted columns
cols = ['grade', 'school', 'percent_frl']
frl_df = frl_df[cols]

# remove excess spacing
frl_df['school'] = frl_df['school'].str.strip()

# convert grade to string for join
frl_df['grade'] = frl_df['grade'].astype(str)

In [4]:
df = prof_df.merge(frl_df, on=['grade', 'school'], how='left')
df.to_csv('./school-poverty-proficency.csv', index=False)